# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902083


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/92 [00:00<?, ?it/s]

Rendering models:   2%|▏         | 2/92 [00:05<03:59,  2.66s/it]

Rendering models:   3%|▎         | 3/92 [00:07<03:31,  2.37s/it]

Rendering models:   4%|▍         | 4/92 [00:07<02:34,  1.75s/it]

Rendering models:   5%|▌         | 5/92 [00:07<01:50,  1.26s/it]

...

Rendering models:  95%|█████████▍| 87/92 [00:17<00:00,  9.36it/s]

Rendering models:  97%|█████████▋| 89/92 [00:17<00:00,  8.62it/s]

Rendering models:  99%|█████████▉| 91/92 [00:17<00:00,  9.00it/s]

Rendering models: 100%|██████████| 92/92 [00:17<00:00,  8.89it/s]

equidad1                              0.089444
kayleebug2017                         0.000862
StephAChamber                         0.000363
Thedunkmasta                          0.002885
brodicius                             0.000437
not-logged-in-0997cfcba8da23453381    0.000195
Thedunkmasta                          0.002686
not-logged-in-846e14e9f3e0f594a568    0.006706
sinjinza1984                          0.000132
Ella7777                              0.000155
graciegirl                            0.000899
msross                                0.000149
clizardo                              0.000119
mkirms                                0.000101
clizardo                              0.000172
msross                                0.000152
djcameron                             0.000135
mchaffin                              0.000193
mchaffin                              0.000198
wthammas                              0.000426
djcameron                             0.000228
atinkess     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())